In [1]:
!pip install gradio

,  Downloading gradio-5.25.2-py3-none-any.whl.metadata (16 kB)
,Collecting aiofiles<25.0,>=22.0 (from gradio)
,  Downloading aiofiles-24.1.0-py3-none-any.whl.metadata (10 kB)
,Requirement already satisfied: anyio<5.0,>=3.0 in /usr/local/lib/python3.11/dist-packages (from gradio) (4.9.0)
,Collecting fastapi<1.0,>=0.115.2 (from gradio)
,  Downloading fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
,Collecting ffmpy (from gradio)
,  Downloading ffmpy-0.5.0-py3-none-any.whl.metadata (3.0 kB)
,Collecting gradio-client==1.8.0 (from gradio)
,  Downloading gradio_client-1.8.0-py3-none-any.whl.metadata (7.1 kB)
,Collecting groovy~=0.1 (from gradio)
,  Downloading groovy-0.1.2-py3-none-any.whl.metadata (6.1 kB)
,Requirement already satisfied: httpx>=0.24.1 in /usr/local/lib/python3.11/dist-packages (from gradio) (0.28.1)
,Requirement already satisfied: huggingface-hub>=0.28.1 in /usr/local/lib/python3.11/dist-packages (from gradio) (0.30.2)
,Requirement already satisfied: jinja2<4.0 in /usr/l

In [2]:
import gradio as gr
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [3]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
,The secret `HF_TOKEN` does not exist in your Colab secrets.
,To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
,You will be able to reuse this secret in all of your notebooks.
,Please note that authentication is recommended but still optional to access public models or datasets.
,  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
,You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [5]:
def transcribe_audio(audio_tensor):
    sample_rate, data = audio_tensor
    audio_tensor = torch.tensor(data)
    audio_tensor = audio_tensor.to(torch.float32)
    if len(audio_tensor.shape) > 1:
        audio_tensor = audio_tensor.mean(dim=0, keepdim=True)
    if audio_tensor.shape[0] < 16000:
        return "Audio too short. Please upload at least 1 second of audio."
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        audio_tensor = resampler(audio_tensor)
        sample_rate=16000
    input_values = processor(audio_tensor.squeeze().numpy(), sampling_rate=16000, return_tensors="pt", padding=True).input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

In [6]:
gr.Interface(fn=transcribe_audio, inputs="audio", outputs="text").launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
,* Running on public URL: https://82ef0581df892fc18b.gradio.live
,
,This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
,Killing tunnel 127.0.0.1:7860 <> https://82ef0581df892fc18b.gradio.live


In [7]:
!jupyter nbconvert --ClearMetadataPreprocessor.enabled=True --inplace --clear-output your_notebook.ipynb


[NbConvertApp] WARNING | pattern 'your_notebook.ipynb' matched no files
,This application is used to convert notebook files (*.ipynb)
,        to various other formats.
,
,        WARNING: THE COMMANDLINE INTERFACE MAY CHANGE IN FUTURE RELEASES.
,
,Options
,=======
,The options below are convenience aliases to configurable class-options,
,as listed in the "Equivalent to" description-line of the aliases.
,To see all configurable class-options for some <cmd>, use:
,    <cmd> --help-all
,
,--debug
,    set log level to logging.DEBUG (maximize logging output)
,    Equivalent to: [--Application.log_level=10]
,--show-config
,    Show the application's configuration (human-readable format)
,    Equivalent to: [--Application.show_config=True]
,--show-config-json
,    Show the application's configuration (json format)
,    Equivalent to: [--Application.show_config_json=True]
,--generate-config
,    generate default config file
,    Equivalent to: [--JupyterApp.generate_config=True]
,-y
,    Ans